In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
from collections import defaultdict
import logging
import pyhmy
from pyhmy import rpc

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
    url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [6]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    return block, last_block

In [7]:
def getEpoch():
    method = 'hmy_getEpoch'
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [8]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [9]:
def getStakingTransactionHistory(address):
    method = 'hmyv2_getStakingTransactionsHistory'
    params = [{"address": address,
               "pageIndex": 0,
               "pageSize": 100000,
               "fullTx": True,
               "txType": "ALL",
               "order": "DESC"}]
    return get_information(method, params)['result']['staking_transactions']

In [10]:
def U2_test(single):
    logger.info(f"Test-U2: After undelegate, the total stake amount for that validator should subtract the undelegation amount before next epoch")
    num = 1
    iterations = 0
    flag = True
    curr_test = None
    while iterations < num:
        block, last_block = getCurrentAndLastBlock()
        first_block = last_block - 16384 + 1
        logger.info(f"current and last block numbers: {block}, {last_block}")
        prev_stake = dict()
        while block < last_block+1:
            block = getBlockNumber()    
        epoch = getEpoch()
        change = False
        logger.info(f"first block {block} of new epoch: {epoch} reached, collecting total delegation...")        
        validator_infos = getAllValidatorInformation()        
        for i in validator_infos:
            address = i['validator']['address']
            prev_stake[address] = i['total-delegation']
            try:
                staking_txs = rpc.account.get_staking_transaction_history(address, page=0, page_size=1000, include_full_tx=True, tx_type='ALL', order='DESC', endpoint='https://api.s0.t.hmny.io/', timeout = 180)
                for txs in staking_txs:
                    txs_block = txs['blockNumber']
                    if txs_block < first_block:
                        break
                    if first_block <= txs_block <= last_block:
                        if txs['type'] == 'Undelegate' and txs['msg']['validatorAddress'] == address:
                            prev_stake[address] -= txs['msg']['amount']
                            change = True
                            logger.info(f"Validator {address} reduce stakes: {txs['msg']['amount']}")   
            except:
                pass
     
        if not change:
            logger.info("Test-U2: Need more tests\n")
            return "Need More Tests", curr_test
        
        logger.info("wait for new epoch")
        new_epoch = getEpoch()
        while epoch == new_epoch:
            new_epoch = getEpoch()
        
        block, last_block = getCurrentAndLastBlock()
        first_block = last_block - 16384 + 1
        while block < last_block+1:
            block = getBlockNumber()
        epoch = getEpoch()
        logger.info(f"first block {block} of new epoch {epoch} reached, will compare the expected stakes and real stakes")
        flag = True
        validator_infos = getAllValidatorInformation()
        for i in validator_infos:
            address = i['validator']['address']
            if address in prev_stake:
                curr_stake = i['total-delegation']
                try: 
                    staking_txs = rpc.account.get_staking_transaction_history(address, page=0, page_size=1000, include_full_tx=True, tx_type='ALL', order='DESC', endpoint='https://api.s0.t.hmny.io/', timeout = 180)
                    expect_stake = prev_stake[address]
                    for txs in staking_txs:
                        txs_block = txs['blockNumber']
                        if txs_block < first_block:
                            break
                        if first_block <= txs_block <= last_block:
                            if txs['type'] == 'Delegate' and txs['msg']['validatorAddress'] == address:
                                expect_stake += txs['msg']['amount']
                                logger.info(f"Validator {address} get more stakes: {txs['msg']['amount']}")

                    if expect_stake != curr_stake:
                        flag = False
                        logger.warning(f"Test-U2: Fail")
                        logger.warning(f"Validator {address} expeced stake: {expect_stake}, real stake: {curr_stake}")
                except:
                    pass

        if flag:
            logger.info("Test-U2: Succeed\n")
            return True, curr_test
        else:
            return False, curr_test
           
    iterations += 1    

In [11]:
U2_test(True)

INFO:economic-test:Test-U2: After undelegate, the total stake amount for that validator should subtract the undelegation amount before next epoch
INFO:economic-test:current and last block numbers: 3523988, 3538943
INFO:economic-test:first block 3538944 of new epoch: 196 reached, collecting total delegation...
INFO:economic-test:Validator one1nef0c8kgy30delcckf87jaegdp7s7ntrknvajj reduce stakes: 10000000000000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj reduce stakes: 6000000000000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj reduce stakes: 300000000000000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj reduce stakes: 14990000000000000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj reduce stakes: 2177481000000000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj reduce stakes: 100000000000000000000000
INFO:economic-tes

INFO:economic-test:Validator one1k87w94sdeztdewl9uahw24ld7sd3w726y4nu0x reduce stakes: 1580000000000000000000
INFO:economic-test:Validator one1aha9g2d6scsyktjgx7wm9jwssxjp6lrtl8959z reduce stakes: 1000000000000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue reduce stakes: 9441000000000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue reduce stakes: 300000000000000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue reduce stakes: 75164289680000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue reduce stakes: 1050000000000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue reduce stakes: 1774464000000000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue reduce stakes: 1134335781000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue reduce stakes: 340400000000000000000000
IN

INFO:economic-test:Validator one1nef0c8kgy30delcckf87jaegdp7s7ntrknvajj get more stakes: 10902000000000000524288
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj get more stakes: 3000000000000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj get more stakes: 1302338406000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj get more stakes: 24039000000000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj get more stakes: 5334000000000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj get more stakes: 25920000000000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj get more stakes: 2036000000000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj get more stakes: 6597520000000000000000
INFO:economic-test:Validator one1kf42rl6yg2avkjsu34ch2jn8yjs64ycn4n9wdj get more stakes: 844200000000

INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 get more stakes: 120000000000000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 get more stakes: 2000000000000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 get more stakes: 240094000000000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 get more stakes: 1232000000000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 get more stakes: 4165369547000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 get more stakes: 4165369547000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 get more stakes: 3953511328000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 get more stakes: 2123640252000000000000
INFO:economic-test:Validator one14wyp73qmn3zch98dqnjvu4rprcz79tlrxq3al6 get more stakes: 31810000000

INFO:economic-test:Validator one1dcmp24uqgwszcvmm8n6r5dvhqhuukdj86pkg6n get more stakes: 6000000000000000000000
INFO:economic-test:Validator one1dcmp24uqgwszcvmm8n6r5dvhqhuukdj86pkg6n get more stakes: 100000000000000000000000
INFO:economic-test:Validator one1dcmp24uqgwszcvmm8n6r5dvhqhuukdj86pkg6n get more stakes: 4863000000000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 get more stakes: 11705623685000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 get more stakes: 101636415418000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 get more stakes: 4300000000000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 get more stakes: 8000000000000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 get more stakes: 2900000000000000000000
INFO:economic-test:Validator one1qk7mp94ydftmq4ag8xn6y80876vc28q7s9kpp7 get more stakes: 6200000000

INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue get more stakes: 10000000000000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue get more stakes: 1000000000000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue get more stakes: 4380000000000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue get more stakes: 1000000000000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue get more stakes: 9000000000000000000000
INFO:economic-test:Validator one1r3kwetfy3ekfah75qaedwlc72npqm2gkayn6ue get more stakes: 57090000000000000000000
INFO:economic-test:Validator one1zdz826ruk0rfp78vg3dw98epwn33ky3cu9udmm get more stakes: 14260000000000000000000
INFO:economic-test:Validator one10z5d98vpm5pvzw32vpma3p70vcdk0ckq0znapk get more stakes: 61811000000000000000000
INFO:economic-test:Validator one10z5d98vpm5pvzw32vpma3p70vcdk0ckq0znapk get more stakes: 14950000000

INFO:economic-test:Validator one15mf5amj7nc8c9zpp0e2wr48em0r2vm02k9m8h2 get more stakes: 279768000000000000000000
INFO:economic-test:Validator one18ky073zdrrmme3fs7h63wyzguuj6a3uukuc3gk get more stakes: 1191231426010146706400
INFO:economic-test:Validator one18ky073zdrrmme3fs7h63wyzguuj6a3uukuc3gk get more stakes: 1738419674418290698420
INFO:economic-test:Validator one18ky073zdrrmme3fs7h63wyzguuj6a3uukuc3gk get more stakes: 11154203871148955967388
INFO:economic-test:Validator one18ky073zdrrmme3fs7h63wyzguuj6a3uukuc3gk get more stakes: 2163910277981750503100
INFO:economic-test:Validator one18ky073zdrrmme3fs7h63wyzguuj6a3uukuc3gk get more stakes: 3067784085211878291056
INFO:economic-test:Validator one18ky073zdrrmme3fs7h63wyzguuj6a3uukuc3gk get more stakes: 11154203871148955967388
INFO:economic-test:Validator one18ky073zdrrmme3fs7h63wyzguuj6a3uukuc3gk get more stakes: 6090334197871349360541
INFO:economic-test:Validator one18ky073zdrrmme3fs7h63wyzguuj6a3uukuc3gk get more stakes: 50000000000

(False, None)